In [3]:
"""
OBJETIVO: 
    - Ejecutar la práctica de Mercado Libre en Jupyter Notebook
CREADO POR: LEONARDO KUFFO
ULTIMA VEZ EDITADO: 02 NOVIEMBRE 2023
"""
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess

class Articulo(Item):
    titulo = Field()
    precio = Field()
    descripcion = Field()

class MercadoLibreCrawler(CrawlSpider):
    name = 'mercadoLibre'

    custom_settings = {
      'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
      'CLOSESPIDER_PAGECOUNT': 20 # Numero maximo de paginas en las cuales voy a descargar items. Scrapy se cierra cuando alcanza este numero
    }

    # Utilizamos 2 dominios permitidos, ya que los articulos utilizan un dominio diferente
    allowed_domains = ['articulo.mercadolibre.com.ec', 'listado.mercadolibre.com.ec']

    start_urls = ['https://listado.mercadolibre.com.ec/animales-mascotas/perros/']

    download_delay = 1

    # Tupla de reglas
    rules = (
        Rule( # REGLA #1 => HORIZONTALIDAD POR PAGINACION
            LinkExtractor(
                allow=r'/_Desde_\d+' # Patron en donde se utiliza "\d+", expresion que puede tomar el valor de cualquier combinacion de numeros
            ), follow=True),
        Rule( # REGLA #2 => VERTICALIDAD AL DETALLE DE LOS PRODUCTOS
            LinkExtractor(
                allow=r'/MEC-' 
            ), follow=True, callback='parse_items'), # Al entrar al detalle de los productos, se llama al callback con la respuesta al requerimiento
    )

    def parse_items(self, response):

        item = ItemLoader(Articulo(), response)
        
        # Utilizo Map Compose con funciones anonimas
        # PARA INVESTIGAR: Que son las funciones anonimas en Python?
        item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))
        item.add_xpath('descripcion', '//div[@class="ui-pdp-description"]/p/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

        soup = BeautifulSoup(response.body) # EN GOOGLE COLAB: Cambiar a --> response.text
        precio = soup.find(class_="andes-money-amount__fraction")
        precio_completo = precio.text.replace('\n', ' ').replace('\r', ' ').replace(' ', '') # texto de todos los hijos
        item.add_value('precio', precio_completo)

        yield item.load_item()

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'ml.json'
})

process.crawl(MercadoLibreCrawler)
process.start()

2023-10-27 21:35:59 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-10-27 21:35:59 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.13, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.4 (main, Jun 20 2023, 17:23:00) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform macOS-13.4.1-arm64-arm-64bit
2023-10-27 21:35:59 [scrapy.addons] INFO: Enabled addons:
[]
2023-10-27 21:35:59 [py.warnings] WARNING: /opt/homebrew/lib/python3.11/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the '

2023-10-27 21:36:02 [charset_normalizer] DEBUG: Encoding detection: utf_8 is most likely the one.
2023-10-27 21:36:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://articulo.mercadolibre.com.ec/MEC-520462963-cocker-spaniel-americano-260-cada-uno-_JM>
{'descripcion': ['ENTREGA INMEDIATA. Lindos cachorritos cocker Spaniel '
                 'Americanos 100% puros vacunados y desparasitados con Carnet '
                 'de vacunas al día, ideales para familias ya que son '
                 'cachorros de compañías para niños y adultos muy '
                 'inteligentes, hay machos y hembras'],
 'precio': ['2'],
 'titulo': ['Cocker Spaniel Americano 260 Cada Uno']}
2023-10-27 21:36:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://articulo.mercadolibre.com.ec/MEC-550031482-cachorros-samoyedo-a330-_JM#position=54&search_layout=stack&type=item&tracking_id=f3622607-925a-4c01-8c2e-af91b9a6746f> (referer: https://listado.mercadolibre.com.ec/animales-mascotas/perros/)
2023-10-2

2023-10-27 21:36:07 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:56: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:07 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:57: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('descripcion', '//div[@class="ui-pdp-description"]/p/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:07 [charset_normalizer] DEBUG: Encoding detection: utf_8 is most likely the one.
2023-10-27 21:36:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://articulo.mercadolibre.com.ec/MEC-516172845-lin

2023-10-27 21:36:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://articulo.mercadolibre.com.ec/MEC-523777288-yorky-cachorrito-lindo-machito-450-buen-precio-yorkshire-_JM#position=47&search_layout=stack&type=item&tracking_id=f3622607-925a-4c01-8c2e-af91b9a6746f> (referer: https://listado.mercadolibre.com.ec/animales-mascotas/perros/)
2023-10-27 21:36:12 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:56: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:12 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:57: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('descripc

2023-10-27 21:36:17 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:56: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:17 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:57: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('descripcion', '//div[@class="ui-pdp-description"]/p/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:17 [charset_normalizer] DEBUG: Encoding detection: utf_8 is most likely the one.
2023-10-27 21:36:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://articulo.mercadolibre.com.ec/MEC-528373432-cac

2023-10-27 21:36:21 [scrapy.core.engine] INFO: Closing spider (closespider_pagecount)
2023-10-27 21:36:21 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:56: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:21 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:57: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('descripcion', '//div[@class="ui-pdp-description"]/p/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:22 [charset_normalizer] DEBUG: Encoding detection: utf_8 is most likely the one.
2023-10-27 21:36:22 [scrapy.core.sc

2023-10-27 21:36:27 [charset_normalizer] DEBUG: Encoding detection: utf_8 is most likely the one.
2023-10-27 21:36:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://articulo.mercadolibre.com.ec/MEC-538386126-bellos-cachorros-chihuahua-450-puros-miniatura-_JM>
{'descripcion': ['TE OFRECEMOS LOS PERRITOS CHIHUAHUA MÀS LINDOS Y PUROS, TE '
                 'GARANTIZAMOS SU GENÈTICA, SUS PADRES SON MINI CABEZA DE '
                 'MANZANA TE LOS ENTREGAMOS EN CUALQUIER CIUDAD DEL PAÌS POR '
                 'CUATROCIENTOS CINCUENTA USD, VACUNADOS DESPARASITADOS '
                 'PUESTOS LA PIPETA ANTIPULGAS Y CON CERTIFICADO DE PUREZA '
                 'PARA GARANTIZARTELA',
                 'ADEMÀS DE LAS RECOMENDACIONES QUE NECESITAS PARA EL CUIDADO '
                 'DE TU NUEVO BEBÈ Y CARACTERÌSTICAS DE LA RAZA POR ESCRITO '
                 'PARA QUE SIGAS EL CRECIMIENTO DE TUS CACHORROS Y MIRES QUE '
                 'SON TOTALMENTE PUROS',
                 'NO DUDES EN

2023-10-27 21:36:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://articulo.mercadolibre.com.ec/MEC-548993422-casa-para-mascotas-casa-razas-pequenas-_JM>
{'descripcion': ['Casa Razas Pequeñas',
                 'Color: Rojo y Beige',
                 'Materia: Polipropileno',
                 'Peso:4,97 Kg',
                 'Tamaño Longitud x Alto x Ancho (Cm): 68 x 58,5 x 61',
                 'Para uso en interiores y exteriores, con protección UV',
                 'Material resistente a la humedad, fácil de limpiar',
                 'Fácil de armar, sin herramientas',
                 'Piso elevado del suelo para aislar y mantener seco al perro',
                 'El diseño del techo evita que la lluvia moje el interior',
                 'Solo quedan 5 disponibles',
                 'Compralo Ya!',
                 'Envios a todo el Ecuador por Servientrega S.A Oficina '
                 ',Retiro en Agencia,Envío a Domicilio.',
                 'El Envío se efectúa una v

2023-10-27 21:36:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://articulo.mercadolibre.com.ec/MEC-550134710-cachorros-salchicha-teckel-a200-_JM#position=44&search_layout=stack&type=item&tracking_id=dafa0030-267f-4c5e-baf1-e2cc9458dcd9> (referer: https://listado.mercadolibre.com.ec/animales-mascotas/perros/_Desde_51_NoIndex_True)
2023-10-27 21:36:36 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:56: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))

2023-10-27 21:36:36 [py.warnings] WARNING: /var/folders/5q/75t4z1sd6vg5lq56p9vh1cgm0000gn/T/ipykernel_83055/2482256198.py:57: ScrapyDeprecationWarning: scrapy.loader.processors.MapCompose is deprecated, instantiate itemloaders.processors.MapCompose instead.
  item.add_xpath('descripcion

2023-10-27 21:36:39 [scrapy.extensions.feedexport] INFO: Stored json feed (33 items) in: ml.json
2023-10-27 21:36:39 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 19337,
 'downloader/request_count': 35,
 'downloader/request_method_count/GET': 35,
 'downloader/response_bytes': 3099519,
 'downloader/response_count': 35,
 'downloader/response_status_count/200': 35,
 'dupefilter/filtered': 51,
 'elapsed_time_seconds': 40.199701,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'closespider_pagecount',
 'finish_time': datetime.datetime(2023, 10, 27, 19, 36, 39, 699485, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 12240768,
 'httpcompression/response_count': 35,
 'item_scraped_count': 33,
 'log_count/DEBUG': 103,
 'log_count/INFO': 11,
 'log_count/WARNING': 68,
 'memusage/max': 118194176,
 'memusage/startup': 118194176,
 'request_depth_max': 3,
 'response_received_count': 35,
 'scheduler/dequeued': 35,
 'scheduler/dequ